# Lyft basics: fetch the data, visualize it

Since I need to get acquainted with the data myself, I thought sharing this notebook for other people who are starting now the competition would be good.

As I always do for this kind of notebooks, I am handling the imports cell by cell, so if you need just a part of what I am describing here it will be easier for you to figure out which imports you need (at least, this is my hope). I make an exception for the most common imports:

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot  as plt # data visualization

First of all, let's pip install l5kit (Level 5 Kit). For this notebook I am relying heavily on their [documentation](https://github.com/lyft/l5kit).

In [ ]:
!pip install --upgrade pip
!pip install pymap3d==2.1.0
!pip install -U l5kit

At this point we need a yaml file to configure the visualization. The one from [l5kit github](https://github.com/lyft/l5kit) will do just fine. If you aren't just copy this notebook, there is already a [dataset](https://www.kaggle.com/jpbremer/lyft-config-files) you can add to your notebook to have the files (or you can download your own copy and add it, but kaggle will suggest you use the existing dataset to avoid having too many copies of the same thing).

In [ ]:
import os
from l5kit.configs import load_config_data

# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"
# get config
cfg = load_config_data("../input/lyft-config-files/visualisation_config.yaml")

\- when loaded in python, the `yaml` file is converted into a python `dict`. 

`raster_params` contains all the information related to the transformation of the 3D world onto an image plane. Let's look into it:


In [ ]:
print(f'current raster_param:\n')
for k,v in cfg["raster_params"].items():
    print(f"{k}:{v}")

  - `raster_size`: the image plane size
  - `pixel_size`: how many meters correspond to a pixel
  - `ego_center`: the raster is centered around an agent, this parameter moves the agent in the image
  - `map_type`: the rasterizer to be employed. The ones currently supported are a satellite-based and a semantic-based one

To read the data, we simply do:

In [ ]:
from l5kit.data import ChunkedDataset, LocalDataManager

dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)

This example shows how `.zarr` files support most of the traditional numpy array operations. 

As an example, here we iterate over the frames to get a scatter plot of the AV locations:

In [ ]:
from tqdm import tqdm

frames = zarr_dataset.frames
coords = np.zeros((len(frames), 2))
for idx_coord, idx_data in enumerate(tqdm(range(len(frames)), desc="getting centroid to plot trajectory")):
    frame = zarr_dataset.frames[idx_data]
    coords[idx_coord] = frame["ego_translation"][:2]

In [ ]:
plt.scatter(coords[:, 0], coords[:, 1], marker='.')
axes = plt.gca()
axes.set_xlim([-1500, 1600])
axes.set_ylim([-2500, 1600])

Let's now move to 

In [ ]:
from l5kit.dataset import (EgoDataset,      # iterates over the AV annotations
                           AgentDataset)    #iterates over other agents annotations

from l5kit.rasterization import build_rasterizer

rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)

# Visualising the Autonomous Vehicle (AV)

In [ ]:
from l5kit.geometry import transform_points

from l5kit.visualization import (draw_trajectory,       # draws 2D trajectories from coordinates and yaws offset on an image
                                 TARGET_POINTS_COLOR)

data = dataset[50]

im = data["image"].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

plt.imshow(im[::-1])
plt.show()

# Visualising an agent

In [ ]:
dataset = AgentDataset(cfg, zarr_dataset, rast)
data = dataset[50]

im = data["image"].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

plt.imshow(im[::-1])
plt.show()

# Visualizing a whole scene

In [ ]:
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_satellite"   # This can be changed to "py_semantic" to have a view similar to the ones before
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 50
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

# Seeing the scene in movement

For completion sake, I will now take some inspiration/code from this other [notebook](https://www.kaggle.com/nxrprime/lyft-understanding-the-data-and-eda) to animate what we just saw

In [ ]:
from matplotlib import animation, rc

def animate_solution(images):

    def animate(i):
        im.set_data(images[i])
 
    fig, ax = plt.subplots()
    im = ax.imshow(images[0])
    
    return animation.FuncAnimation(fig, animate, frames=len(images), interval=60)

In [ ]:
from IPython.display import HTML

scene_idx = 50
indexes = dataset.get_scene_indices(scene_idx)

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    images.append(PIL.Image.fromarray(im[::-1]))
    
anim = animate_solution(images)
HTML(anim.to_jshtml())

# Work in progress - Will continue